In [20]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import sys,os
sys.path.append(os.getcwd())
from descriptive_analysis import *
import random_forest as rf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Building an Optimal Premium Model in an insurance company #

## Problem description ##

We are interested in solving a CRM problem for an insurance company. The tasks to be achieved are:

* Finding the ideal target, in this case, people who are more likely to contract the insurance.
* Obtaining the premium we should offer to each client, it means, the optimal price that should be offered to the clients.
* Calculating the difference between offering the premium randomly and optimally using the information obtained in the model.


## Working with data ##

Two databases with clients’ information are available.

In the first one we have the information of 20.000 clients which have already been contacted; 9% of them have contracted the product.
 
Important data is included such as the premium offered, the number of products that they have already bought, the number of years that they have been clients of the company and the socioeconomic status (an economic and sociological measure combined with the person’s work experience and its individual’s or family’s economic and social position in relation to others, based on income, education, and occupation).

In the second database of non-previously contacted clients, we have the same information of 10.000 clients but only 5.000 are going to be contacted due to mechanical restrictions.

Is worthwhile offering the same premium to all the clients? Is better focusing on people with some characteristics than choosing the clients randomly? 

In [97]:
xls = pd.ExcelFile('Database.xlsx')
variable_description = xls.parse(0)
db1 = xls.parse(1)
db2 = xls.parse(2)

In [98]:
variable_description

,Variable Name,Meaning
0,Obs,Number of Observations
1,Sales,It indicates whether the client bought a produ...
2,Price Sensitivity,It indicates the client's sensitivity to the p...
3,PhoneType,Client's phone type: Fixed or Mobile
4,Email,It indicates whether the client's email is ava...
5,Tenure,Client's tenure (year when the person became a...
6,NumberofCampaigns,Number of times the client has been called
7,ProdActive,Number of active products
8,ProdBought,Number of different products previously bought
9,Premium Offered,Premium offered to the client


In [99]:
db1.describe()

/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning:

Invalid value encountered in percentile



,Obs,Sales,Price Sensitivity,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Premium Offered,Birthdate,...,Income,yearBuilt,House Insurance,Pension Plan,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices
count,20000.000000,20000.000000,1475.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,9512.000000,...,1.486000e+04,14865.000000,14860.000000,1.486000e+04,1.486000e+04,1.484500e+04,7164.000000,7164.000000,7164.000000,7164.000000
mean,10000.500000,0.085750,3.792542,0.074450,2007.118950,3.709200,0.050700,0.320200,13.831877,1966.409062,...,5.802174e+04,1979.502657,7364.270664,3.728715e+04,1.696786e+04,3.055875e+04,1.504467,1.005444,0.502233,0.503769
std,5773.647028,0.280002,1.694535,0.262508,6.715032,4.156429,0.238186,0.706397,2.774808,11.478364,...,6.644060e+04,23.073381,8542.363258,4.239543e+04,1.945761e+04,7.872496e+04,1.120473,0.817475,0.500030,0.500021
min,1.000000,0.000000,1.000000,0.000000,1990.000000,2.000000,0.000000,0.000000,11.120000,1944.000000,...,2.190805e+03,1900.000000,186.013800,1.661723e+03,6.173887e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,NaN,0.000000,2004.000000,2.000000,0.000000,0.000000,11.120000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,10000.500000,0.000000,NaN,0.000000,2010.000000,2.000000,0.000000,0.000000,14.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,15000.250000,0.000000,NaN,0.000000,2012.000000,4.000000,0.000000,0.000000,14.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,20000.000000,1.000000,6.000000,1.000000,2013.000000,32.000000,3.000000,6.000000,21.850000,1984.000000,...,4.106372e+06,2012.000000,527866.417800,2.620520e+06,1.202556e+06,4.884174e+06,3.000000,2.000000,1.000000,1.000000


In [100]:
db2.describe()

/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning:

Invalid value encountered in percentile



,Obs,Price Sensitivity,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Birthdate,Living Area (m^2),House Price,Income,yearBuilt,House Insurance,Pension Plan,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices
count,10000.00000,2694.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,4541.000000,7831.000000,7.830000e+03,7.831000e+03,7831.000000,7831.000000,7.831000e+03,7.831000e+03,7.831000e+03,3760.000000,3760.000000,3760.000000,3760.000000
mean,5000.50000,3.399777,0.046900,2011.248300,3.571400,0.022300,0.24210,1966.269764,203.416630,2.198855e+05,6.002617e+04,1978.753288,7648.644968,3.849267e+04,1.756162e+04,3.229642e+04,1.520479,0.992287,0.509309,0.499202
std,2886.89568,1.766382,0.211435,5.019897,4.237747,0.156221,0.64802,11.597210,597.405984,3.112247e+05,8.546163e+04,23.229961,10987.924223,5.453266e+04,2.502805e+04,8.328343e+04,1.116566,0.813359,0.499980,0.500066
min,1.00000,1.000000,0.000000,1950.000000,2.000000,0.000000,0.00000,1944.000000,0.000000,7.057584e+03,1.580315e+03,1900.000000,134.177400,1.198663e+03,4.453454e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,2500.75000,NaN,0.000000,2012.000000,2.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5000.50000,NaN,0.000000,2012.000000,2.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7500.25000,NaN,0.000000,2013.000000,4.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,10000.00000,6.000000,1.000000,2013.000000,34.000000,2.000000,5.00000,1984.000000,22943.212000,1.611964e+07,4.426344e+06,2012.000000,569032.313400,2.824619e+06,1.296269e+06,2.998083e+06,3.000000,2.000000,1.000000,1.000000


 ## Descriptive analysis ##
 
 With all the data available in the first database is extremely important to make a complete descriptive analysis of the variables to understand the type of information we are dealing with, which can give us an idea of which variables are relevants to solve our problem. 

For every single variable, we will run a single logistic regression to analyze its importance, and select only a subset of the features for our final model (there is a danger in this method: we may lose any effect due not to individual variables, but to the relationship between them).

Our objective is to maximize the profit, so our objective is not going to be the Sales, but the 'Number of Semesters Paid' (a sales that does not reflect later in payments is of no use to us)

In [101]:
db1.loc[np.isnan(db1['Number of Semesters Paid']), 'Number of Semesters Paid'] = 0
print(db1['Number of Semesters Paid'].mean())
print(db1['Number of Semesters Paid'].std())
print((db1['Number of Semesters Paid'] > 0.5).mean())
print(db1['Premium Offered'].mean())
scores = {}

0.1974
0.8560489945054709
0.05405
13.831877499999042


## Total sales ##

We want ot optimize the total sales, that is the amount generated by the client paying the premium over several semesters (many of the positive Sales do not convert later in payments, so this is not the most fundamenta variable).

In [106]:
db1['TotalSales'] = db1['Premium Offered'] * db1['Number of Semesters Paid']

## Variable importance ##

A way of determining the variable importance, before selecting the most important for our model, is to adjust a random forest model with all our data.

In [107]:
X_train, X_test, y_train, y_test = rf.prepare_data()
model = rf.random_forest(file=r'rf_data.dat')

Training...
Test mean squared error: 57.654


In [108]:
features = ['']
importances = model.feature_importances_

features = [x for x in db1.columns if x not in rf.not_features]
features.remove('Premium Offered')
features = ['Premium Offered'] + features



In [109]:
features = [x for x in db1.columns if x not in rf.not_features]
features.remove('Premium Offered')
features = ['Premium Offered'] + features
feature_importance = pd.Series(index=features, data=model.feature_importances_)
feature_importance = feature_importance.sort_values(ascending=False)
feature_importance

ProdActive                         0.475221
NumberofCampaigns                  0.107191
Birthdate                          0.051074
ProdBought                         0.040778
Province                           0.040051
Living Area (m^2)                  0.038102
Tenure                             0.035284
yearBuilt                          0.029217
Premium Offered                    0.020982
Pension Plan                       0.015392
House Price                        0.015136
Income                             0.014230
Email                              0.013971
Credit                             0.013345
Socieconomic Status                0.012078
House Insurance                    0.011796
Savings                            0.010735
Right Address                      0.010035
Price Sensitivity                  0.009321
Number of Mobile Phones            0.008057
PhoneType                          0.007029
Type of House                      0.006927
Number of Fixed Lines           

In [110]:
dp = [
    # Chart
    go.Bar(
        x=feature_importance.index,
        y=feature_importance.values,
        name='Feature importance')
    ]

layout = go.Layout(
        barmode='group',
        title='Feature importance',
        yaxis=dict(title='Feature importance'),
        xaxis=dict(title='Features',         
                    tickfont=dict(size=9,),
                  ),
    )

fig = go.Figure(data=dp, layout=layout)
iplot(fig)

We we select for our analysis all variables until 'Price Sensitivity'. This is totally arbitrary, but we are intereseted in conserve this variable, and it's as good as any other possible cut.

In [111]:
selected_features = feature_importance.index[feature_importance >= feature_importance['Price Sensitivity']]
selected_features

Index(['ProdActive', 'NumberofCampaigns', 'Birthdate', 'ProdBought',
       'Province', 'Living Area (m^2)', 'Tenure', 'yearBuilt',
       'Premium Offered', 'Pension Plan', 'House Price', 'Income', 'Email',
       'Credit', 'Socieconomic Status', 'House Insurance', 'Savings',
       'Right Address', 'Price Sensitivity'],
      dtype='object')

### Price sensitivity ###

It indicates the client's sensitivity to the price: 1 (less sensitive) - 6 (more sensitive) 

The clients without data in the *Price Sensitivity* feature have a lower probability of buying the insurance, so we have included them in the analysis with the value 7, making them the more sensible to the price

In [112]:
var = 'Price Sensitivity'
db1.loc[np.isnan(db1[var]), var] = 7
chi, pval, fig = analyze_feature(var, db1)
scores[var] = (chi, pval)
iplot(fig)

### PhoneType ###

Client's phone type: Fixed or Mobile

In [113]:
chi, pval, fig = analyze_feature('PhoneType', db1, categorical=True)
scores['PhoneType'] = chi, pval
iplot(fig)

### Email ###

It indicates whether the client's email is available: 1 (yes), 0 (otherwise)

In [114]:
chi, pval, fig = analyze_feature('Email', db1)
scores['Email'] = (chi, pval)
iplot(fig)

### Tenure ###

Client's tenure (year when the person became a client of the company)

In [115]:
db1['TenureYears'] = 2013 - db1['Tenure']
chi, pval, fig = analyze_feature('TenureYears', db1, categorical=False)
scores['TenureYears'] = (chi, pval)
iplot(fig)

### Number of Campaigns ###

Number of times the client has been called

In [116]:
chi, pval, fig = analyze_feature('NumberofCampaigns', db1, categorical=False)
scores['NumberofCampaigns'] = (chi, pval)
iplot(fig)

### ProdActive ###

Number of active products

In [117]:
chi, pval, fig = analyze_feature('ProdActive', db1, categorical=False)
scores['ProdActive'] = (chi, pval)
iplot(fig)

### ProdBought ###

Number of different products previously bought

In [118]:
chi, pval, fig = analyze_feature('ProdBought', db1, categorical=False)
scores['ProdBought'] = (chi, pval)
iplot(fig)

### Premium Offered ###

Premium offered to the client

In [119]:
chi, pval, fig = analyze_feature('Premium Offered', db1, categorical=False)
scores['Premium Offered'] = (chi, pval)
iplot(fig)

### Phone Call Day ###

Day the phone call is received

In [208]:
chi, pval, fig = analyze_feature('Phone Call Day', db1, categorical=True)
scores['Premium Offered'] = (chi, pval)
iplot(fig)

### CodeCategory ###

Category of the phone call answer

In [209]:
fig = analyze_output('CodeCategory', db1)
iplot(fig)

### Socieconomic Status ###

It indicates the client's socieconomic status

In [210]:
db1.loc[pd.isnull(db1['Socieconomic Status']), 'Socieconomic Status'] = 0
db1.loc[db1['Socieconomic Status'] == 'Low', 'Socieconomic Status'] = 1
db1.loc[db1['Socieconomic Status'] == 'Medium', 'Socieconomic Status'] = 2
db1.loc[db1['Socieconomic Status'] == 'High', 'Socieconomic Status'] = 3
db1.loc[db1['Socieconomic Status'] == 'Very High', 'Socieconomic Status'] = 4

chi, pval, fig = analyze_feature('Socieconomic Status', db1, categorical=False)
scores['Socieconomic Status'] = (chi, pval)
iplot(fig)

### Province ###

Province where the client lives

In [211]:
db1.loc[pd.isnull(db1['Province']), 'Province'] = 'Unknown'
chi, pval, fig = analyze_feature('Province', db1, categorical=True)
scores['Province'] = (chi, pval)
iplot(fig)

### Right Address ###

It indicates whether the client's address is correct: 1 (yes), 0 (otherwise)

In [212]:
db1.loc[pd.isnull(db1['Right Address']), 'Right Address'] = 'Wrong'
chi, pval, fig = analyze_feature('Right Address', db1, categorical=True)
scores['Right Address'] = (chi, pval)
iplot(fig)

In [213]:
var = 'Living Area (m^2)'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### House Price ###

Estimated price of the house

In [214]:
var = 'House Price'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### yearBuilt ###

It indicates when the client's house was built

In [215]:
var = 'yearBuilt'
db1.loc[pd.isnull(db1[var]), var] = db1[var].median()
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### House Insurance ###

Price of the house insurance

In [216]:
var = 'House Insurance'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### Pension Plan ###

Estimated amount of money the client would have in a pension plan

In [217]:
var = 'Pension Plan'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### Estimated number of cars ###

Estimation of the number of cars owned by the client 

In [218]:
var = 'Estimated number of cars'
db1.loc[pd.isnull(db1[var]), var] = 0
db1.loc[db1[var] == 'None', var] = 0
db1.loc[db1[var] == 'One', var] = 1
db1.loc[db1[var] == 'two', var] = 2
db1.loc[db1[var] == 'Three', var] = 3
chi, pval, fig = analyze_feature(var, db1, categorical=False)
scores[var] = (chi, pval)
iplot(fig)

### Probability of Second Residence ###
Probability of having a second residence

In [219]:
var = 'Probability of Second Residence'
db1.loc[pd.isnull(db1[var]), var] = 0
db1.loc[db1[var] == 'Low', var] = 0
db1.loc[db1[var] == 'Medium', var] = 1
db1.loc[db1[var] == 'High', var] = 2
chi, pval, fig = analyze_feature(var, db1, categorical=False)
scores[var] = (chi, pval)
iplot(fig)

### Credit ###

Estimation of the amount of credit that could be offered to the client

In [220]:
var = 'Credit'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### Savings ###
Estimation of the amount of money saved by the client

In [221]:
var = 'Savings'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### Number of Mobile Phones ###
Number of mobile phones

In [222]:
var = 'Number of Mobile Phones'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### Number of Fixed Lines ###

Number of Land Lines

In [223]:
var = 'Number of Fixed Lines'
db1.loc[pd.isnull(db1[var]), var] = 0
chi, pval, fig = analyze_feature(var, db1, continous=False, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

### ADSL ###

It indicates whether the client has ADSL:1 (yes), 0 (otherwise)

In [224]:
var = 'ADSL'
db1.loc[pd.isnull(db1[var]), var] = 0.5
chi, pval, fig = analyze_feature(var, db1, continous=False, categorical=False)
scores[var] = (chi, pval)
iplot(fig)

### 3G Devices ###
It indicates whether the client has 3G Devices:1 (yes), 0 (otherwise)

In [225]:
var = '3G Devices'
db1.loc[pd.isnull(db1[var]), var] = 0.5
chi, pval, fig = analyze_feature(var, db1, continous=False, categorical=False)
scores[var] = (chi, pval)
iplot(fig)

### Type of House ###

Type of house: Urban or Rural

In [226]:
var = 'Type of House'
db1.loc[pd.isnull(db1[var]), var] = 'Unknown'
chi, pval, fig = analyze_feature(var, db1, continous=False, categorical=True)
scores[var] = (chi, pval)
iplot(fig)

## Price sensivity * Premium ##

Synthetic variable combining the price sensitivy and the premium

In [239]:
var = 'PricePremium'
db1['PricePremium'] = np.exp(-db1['Price Sensitivity'] * db1['Premium Offered'])
chi, pval, fig = analyze_feature(var, db1, continous=True, categorical=False)
scores[var] = (chi, pval)
iplot(fig)

### Feature importance ###

Comparation of feature importance using the chi2 metric

In [240]:
sorted_scores = sorted([(np.log(value[0]),value[1], key) for (key,value) in scores.items()], reverse=True)

dp = [
    # Chart
    go.Bar(
        x=[s[2] for s in sorted_scores],
        y=[s[0] for s in sorted_scores],
        text=['p-value:{:.3f}'.format(p) for p in [s[1] for s in sorted_scores]],
        name='Feature importance')
    ]

layout = go.Layout(
        barmode='group',
        title='Feature importance',
        yaxis=dict(title='log(Chi^2)'),
        xaxis=dict(title='Features',         
                    tickfont=dict(size=9,),
                  ),
    )

fig = go.Figure(data=dp, layout=layout)
iplot(fig)

We will select only the features that have a positive log(chi^2) (this is a totally arbitrary cut)

In [241]:
features = [x[2] for x in sorted_scores if x[0] > 0]
features

['ProdActive',
 'ProdBought',
 'NumberofCampaigns',
 'Email',
 'Province',
 'TenureYears',
 'Socieconomic Status',
 'Price Sensitivity',
 'Right Address',
 'PricePremium',
 'PhoneType',
 'Premium Offered',
 'Estimated number of cars',
 'Living Area (m^2)',
 'Number of Fixed Lines',
 'yearBuilt',
 'Credit',
 'Probability of Second Residence']